In [ ]:
"""
This notebook is used to create input pages for `classify_scanned_page.ipynb` from original reports.

Instructions:

Place the original reports (PDFs) in `data/input/`  
   - The folder should contain PDF files from which pages will be extracted.

Specify the following parameters:  
   - filename: Name of the PDF file to extract pages from.  
   - wanted_page`: The page number to extract.  
   - out_dir`: The target directory where the extracted page will be stored.

Set out_dir to one of the following categories:  
   - "boreprofile"  
   - "maps" 
   - "text" 
   - "title_page"  

The extracted pages are used as **training data** for classification.
"""

import os
import pymupdf
from PIL import Image
import sys

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(repo_root, "src"))


filename = "43283.pdf"

pdf_path = os.path.join(repo_root, "data/input", filename)
out_dir = os.path.join(repo_root, "data/input/boreprofile")

wanted_page = 41

with pymupdf.open(pdf_path) as doc:
    out_path= os.path.join(out_dir,f"{ os.path.splitext(filename)[0]}_{wanted_page}.pdf")
    new_doc = pymupdf.open()
    for page_index,page in enumerate(doc):
        page_number = page_index +1

        if page_number == wanted_page:
            # pixmap = page.get_pixmap()
            # image = Image.frombytes("RGB", [pixmap.width, pixmap.height], pixmap.samples)
            # image.show()
            
            new_doc.insert_pdf(doc, from_page=wanted_page-1, to_page=wanted_page-1)
            new_doc.save(out_path)
            new_doc.close()
    print(f"saved to {out_path}")

saved to /home/lillemor/Documents/lgd-utils/asset-data-extraction/data/input/boreprofile/43283_41.pdf
